## Load data and check data structure (colab environment)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls drive/MyDrive/data\ -\ MoMD

meta_Office_Products.jsonl  Office_Products.jsonl


In [3]:
import pandas as pd

PRODUCTS_PATH = 'drive/MyDrive/data - MoMD'

meta_df = pd.read_json(PRODUCTS_PATH + '/meta_Office_Products.jsonl', lines=True, chunksize=10000).__next__()

In [4]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   main_category    9910 non-null   object 
 1   title            10000 non-null  object 
 2   average_rating   10000 non-null  float64
 3   rating_number    10000 non-null  int64  
 4   features         10000 non-null  object 
 5   description      10000 non-null  object 
 6   price            5215 non-null   float64
 7   images           10000 non-null  object 
 8   videos           10000 non-null  object 
 9   store            9881 non-null   object 
 10  categories       10000 non-null  object 
 11  details          10000 non-null  object 
 12  parent_asin      10000 non-null  object 
 13  bought_together  0 non-null      float64
 14  subtitle         47 non-null     object 
 15  author           12 non-null     object 
dtypes: float64(3), int64(1), object(12)
memory usage: 1.2+ MB


In [5]:
meta_df.head(1)

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Office Products,Alliance Rubber 07706 Non-Latex Brites File Ba...,4.5,665,[REUSABLE: These colored rubber bands are stre...,[Alliance Rubber Brites File Bands are durable...,2.68,[{'thumb': 'https://m.media-amazon.com/images/...,[{'title': 'AMUU rubber bands Large 50 Pack 5 ...,Alliance,"[Office Products, Office & School Supplies, Ta...",{'Manufacturer': 'Alliance Rubber Company Inc....,B001S28Q4Q,NaN,NaN,NaN


In [6]:
review_df = pd.read_json(PRODUCTS_PATH + '/Office_Products.jsonl', lines=True, chunksize=10000).__next__()

In [7]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   rating             10000 non-null  int64         
 1   title              10000 non-null  object        
 2   text               10000 non-null  object        
 3   images             10000 non-null  object        
 4   asin               10000 non-null  object        
 5   parent_asin        10000 non-null  object        
 6   user_id            10000 non-null  object        
 7   timestamp          10000 non-null  datetime64[ns]
 8   helpful_vote       10000 non-null  int64         
 9   verified_purchase  10000 non-null  bool          
dtypes: bool(1), datetime64[ns](1), int64(2), object(6)
memory usage: 713.0+ KB


In [8]:
review_df.head(1)

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5,Pretty & I love it!,Lovely ink. Writes well. The right amount of w...,[],B01AHHL4X2,B01MZ3SD2X,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2023-03-04 14:15:45.945,0,True


## Load spark from jsonl

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import get_json_object, from_json, regexp_replace, col
from pyspark.sql.types import *

spark = SparkSession.builder.appName('Office').getOrCreate()

review_df = spark.read.json(PRODUCTS_PATH + '/Office_Products.jsonl')

In [10]:
meta_schema = StructType([
    StructField("main_category", StringType(), True),
    StructField("title", StringType(), True),
    StructField("average_rating", DoubleType(), True),
    StructField("rating_number", LongType(), True),
    StructField("features", ArrayType(StringType()), True),
    StructField("description", ArrayType(StringType()), True),
    StructField("price", DoubleType(), True),
    StructField("images", ArrayType(MapType(StringType(), StringType())), True),
    StructField("videos", ArrayType(MapType(StringType(), StringType())), True),
    StructField("store", StringType(), True),
    StructField("categories", ArrayType(StringType()), True),

    StructField("details", StringType(), True),

    StructField("parent_asin", StringType(), True),
    StructField("bought_together", DoubleType(), True),
    StructField("subtitle", StringType(), True),
    StructField("author", StringType(), True),
])

raw_meta_df = spark.read.text(PRODUCTS_PATH + "/meta_Office_Products.jsonl")

meta_df = (
    raw_meta_df
    .select(from_json(col("value"), meta_schema).alias("data"))
    .select("data.*")
)

## Show schema

In [11]:
meta_df.printSchema()

root
 |-- main_category: string (nullable = true)
 |-- title: string (nullable = true)
 |-- average_rating: double (nullable = true)
 |-- rating_number: long (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- description: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- price: double (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- videos: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- store: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- details: string (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- bought_together: double (nullable = true)
 |-- subtitle: string (nu

In [12]:
review_df.printSchema()

root
 |-- asin: string (nullable = true)
 |-- helpful_vote: long (nullable = true)
 |-- images: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- attachment_type: string (nullable = true)
 |    |    |-- large_image_url: string (nullable = true)
 |    |    |-- medium_image_url: string (nullable = true)
 |    |    |-- small_image_url: string (nullable = true)
 |-- parent_asin: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- text: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- title: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- verified_purchase: boolean (nullable = true)



## Data quality

### Check null

In [13]:
total_review = review_df.count()
total_meta = meta_df.count()

In [14]:
from pyspark.sql.functions import col, when, count, lit, concat, round, sum

null_review_counts = review_df.select([
    sum(
        col(c).isNull().cast('int')
    ).alias(c)
    for c in review_df.columns
])

null_review_counts.select([
    concat(
        col(c).cast('string'),
        lit(" ("),
        round(col(c).cast('float') * 100 / total_review, 2).cast('string'),
        lit("%)")
    ).alias(c)
    for c in review_df.columns
]).show()

+--------+------------+--------+-----------+--------+--------+---------+--------+--------+-----------------+
|    asin|helpful_vote|  images|parent_asin|  rating|    text|timestamp|   title| user_id|verified_purchase|
+--------+------------+--------+-----------+--------+--------+---------+--------+--------+-----------------+
|0 (0.0%)|    0 (0.0%)|0 (0.0%)|   0 (0.0%)|0 (0.0%)|0 (0.0%)| 0 (0.0%)|0 (0.0%)|0 (0.0%)|         0 (0.0%)|
+--------+------------+--------+-----------+--------+--------+---------+--------+--------+-----------------+



In [15]:
null_meta_counts = meta_df.select([
    sum(
        col(c).isNull().cast('int')
    ).alias(c)
    for c in meta_df.columns
])

null_meta_counts.select([
    concat(
        col(c).cast('string'),
        lit(" ("),
        round(col(c).cast('float') * 100 / total_meta, 2).cast('string'),
        lit("%)")
    ).alias(c)
    for c in meta_df.columns
]).show()

+-------------+--------+--------------+-------------+--------+-----------+---------------+--------+--------+-------------+----------+--------+-----------+---------------+---------------+--------------+
|main_category|   title|average_rating|rating_number|features|description|          price|  images|  videos|        store|categories| details|parent_asin|bought_together|       subtitle|        author|
+-------------+--------+--------------+-------------+--------+-----------+---------------+--------+--------+-------------+----------+--------+-----------+---------------+---------------+--------------+
|23944 (3.37%)|0 (0.0%)|      0 (0.0%)|     0 (0.0%)|0 (0.0%)|   0 (0.0%)|428529 (60.31%)|0 (0.0%)|0 (0.0%)|12251 (1.72%)|  0 (0.0%)|0 (0.0%)|   0 (0.0%)|710503 (100.0%)|702873 (98.93%)|709069 (99.8%)|
+-------------+--------+--------------+-------------+--------+-----------+---------------+--------+--------+-------------+----------+--------+-----------+---------------+---------------+------

### Check NaN

In [18]:
review_numeric_fields = ['rating', 'helpful_vote']

na_review_counts = review_df.select([
    sum(
        col(c).isNaN().cast('int')
    ).alias(c)
    for c in review_numeric_fields
])

na_review_counts.select([
    concat(
        col(c).cast('string'),
        lit(" ("),
        round(col(c).cast('float') * 100 / total_review).cast('string'),
        lit("%)")
    ).alias(c)
  for c in na_review_counts.columns
]).show()

+--------+------------+
|  rating|helpful_vote|
+--------+------------+
|0 (0.0%)|    0 (0.0%)|
+--------+------------+



In [19]:
meta_numeric_fields = ['average_rating', 'rating_number', 'price', 'bought_together']

na_meta_counts = meta_df.select([
    sum(
        col(c).isNaN().cast('int')
    ).alias(c)
    for c in meta_numeric_fields
])

na_meta_counts.select([
    concat(
        col(c).cast('string'),
        lit(" ("),
        round(col(c).cast('float') * 100 / total_meta).cast('string'),
        lit("%)")
    ).alias(c)
    for c in na_meta_counts.columns
]).show()

+--------------+-------------+--------+---------------+
|average_rating|rating_number|   price|bought_together|
+--------------+-------------+--------+---------------+
|      0 (0.0%)|     0 (0.0%)|0 (0.0%)|       0 (0.0%)|
+--------------+-------------+--------+---------------+



### Check empty list

In [24]:
from pyspark.sql.functions import size

review_list_fields = ['images']

empty_review_counts = review_df.select([
    sum(
        (size(col(c)) == 0).cast('int')
    ).alias(c)
    for c in review_list_fields
])

empty_review_counts.select([
    concat(
        col(c).cast('string'),
        lit(" ("),
        round(col(c).cast('float') * 100 / total_review, 2).cast('string'),
        lit("%)")
    ).alias(c)
    for c in empty_review_counts.columns
]).show()

+-----------------+
|           images|
+-----------------+
|12149163 (94.58%)|
+-----------------+



In [25]:
from pyspark.sql.functions import size

meta_list_fields = ['features', 'description', 'images', 'videos', 'categories']

empty_meta_counts = meta_df.select([
    sum(
        (size(col(c)) == 0).cast('int')
    ).alias(c)
    for c in meta_list_fields
])

empty_meta_counts.select([
    concat(
        col(c).cast('string'),
        lit(" ("),
        round(col(c).cast('float') * 100 / total_review, 2).cast('string'),
        lit("%)")
    ).alias(c)
    for c in empty_meta_counts.columns
]).show()

+--------------+--------------+------------+--------------+-------------+
|      features|   description|      images|        videos|   categories|
+--------------+--------------+------------+--------------+-------------+
|159860 (1.24%)|288566 (2.25%)|3266 (0.03%)|505312 (3.93%)|75657 (0.59%)|
+--------------+--------------+------------+--------------+-------------+



### Meta parent_asin duplications

In [26]:
total_meta != meta_df.select('parent_asin').distinct().count()

False